<a href="https://colab.research.google.com/github/okayteakay/Fault-Analysis/blob/main/Fault_Analysis_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel('fault_dataset_new.xlsx')


In [3]:
df.head()

,RMS,Mean,fault in switch
0,58.703351,-33.734571,1
1,78.513080,-44.759827,1
2,84.864746,-48.919557,1
3,103.667227,-59.331936,1
4,110.626172,-63.883449,1


In [4]:
df.describe()

,RMS,Mean,fault in switch
count,432.000000,4.320000e+02,432.000000
mean,115.410739,-1.174613e-02,3.500000
std,36.616542,3.321025e+01,2.293944
min,23.874025,-8.238858e+01,0.000000
25%,88.312074,-1.103781e+01,1.750000
50%,116.501125,-8.790549e-07,3.500000
75%,143.389474,1.103781e+01,5.250000
max,196.885243,8.135974e+01,7.000000


In [5]:
for c in df.columns[:-1]:
  mu = df[c].mean()
  sigma = df[c].std()
  df[c] = (df[c] - mu)/(sigma)
print(df.head())

        RMS      Mean  fault in switch
0 -1.548682 -1.015434                1
1 -1.007677 -1.347418                1
2 -0.834213 -1.472672                1
3 -0.320716 -1.786201                1
4 -0.130667 -1.923253                1


In [6]:
for c in df.columns[:-1]:
  min = df[c].min()
  max = df[c].max()
  df[c] = (df[c] - min)/(max - min)
print(df.head())

        RMS      Mean  fault in switch
0  0.201313  0.297127                1
1  0.315812  0.229796                1
2  0.352525  0.204393                1
3  0.461202  0.140805                1
4  0.501425  0.113010                1


In [7]:
y = df['fault in switch']

In [8]:
X = df.drop('fault in switch', axis =1)

In [9]:
model = Sequential()

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(8, activation='softmax'))

print(model)

In [81]:
ACCURACY_THRESHOLD = 0.85

class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('val_accuracy') > ACCURACY_THRESHOLD):
			print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
			self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()

In [82]:
model.compile(loss = 'categorical_crossentropy',optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),metrics = ['accuracy'])

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=91)

In [84]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
X_val = np.asarray(X_val).astype(np.float32)
y_val = np.asarray(y_val).astype(np.float32)


In [85]:
print(type(X_train), type(y_train), type(X_val), type(y_val))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [86]:
y_train_one_hot = to_categorical(y_train, 8)
y_val_one_hot = to_categorical(y_val, 8)
y_test_one_hot = to_categorical(y_test, 8)


In [87]:
hist = model.fit(X_train,y_train_one_hot, epochs=3000, batch_size = 32, validation_data=(X_val, y_val_one_hot), verbose=1, callbacks=[callbacks])

Epoch 1/3000
9/9 [==============================] - 1s 27ms/step - loss: 0.6967 - accuracy: 0.7247 - val_loss: 0.3897 - val_accuracy: 0.8161
Epoch 2/3000
9/9 [==============================] - 0s 7ms/step - loss: 0.5862 - accuracy: 0.7489 - val_loss: 0.3955 - val_accuracy: 0.8161
Epoch 3/3000
9/9 [==============================] - 0s 7ms/step - loss: 0.5755 - accuracy: 0.7135 - val_loss: 0.3958 - val_accuracy: 0.8161
Epoch 4/3000
9/9 [==============================] - 0s 7ms/step - loss: 0.6692 - accuracy: 0.6862 - val_loss: 0.4034 - val_accuracy: 0.7931
Epoch 5/3000
9/9 [==============================] - 0s 6ms/step - loss: 0.6342 - accuracy: 0.6862 - val_loss: 0.4086 - val_accuracy: 0.7816
Epoch 6/3000
9/9 [==============================] - 0s 6ms/step - loss: 0.5321 - accuracy: 0.7618 - val_loss: 0.4053 - val_accuracy: 0.7816
Epoch 7/3000
9/9 [==============================] - 0s 6ms/step - loss: 0.5984 - accuracy: 0.6819 - val_loss: 0.3993 - val_accuracy: 0.8046
Epoch 8/3000
9/9 [=

In [88]:
predictions = model.predict(X_test)

In [70]:
predictions

array([[1.03021264e-07, 2.32034558e-33, 8.24974675e-04, 3.84406239e-01,
        9.60537718e-21, 6.14767909e-01, 1.86575301e-32, 7.46305659e-07],
       [9.03927473e-14, 0.00000000e+00, 9.50458070e-06, 3.44598234e-01,
        1.55419811e-35, 6.55392230e-01, 0.00000000e+00, 8.17193709e-12],
       [0.00000000e+00, 0.00000000e+00, 4.46087179e-05, 9.20681596e-01,
        0.00000000e+00, 7.92738721e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.05568181e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.23128274e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.92809937e-07, 0.00000000e+00, 0.00000000e+00,
        9.89457726e-01, 0.00000000e+00, 1.05418768e-02, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 6.58649000e-24,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.

In [71]:
a = np.argmax(predictions, axis = 1)

In [72]:
y_test

array([3., 5., 3., 1., 1., 4., 2., 1., 4., 2., 1., 4., 6., 3., 6., 6., 0.,
       7., 0., 0., 5., 1., 0., 1., 4., 2., 6., 2., 0., 2., 5., 2., 6., 1.,
       7., 7., 1., 3., 4., 6., 6., 7., 3., 1., 5., 2., 7., 5., 4., 5., 2.,
       3., 2., 6., 7., 1., 3., 6., 7., 4., 3., 7., 5., 3., 4., 0., 1., 3.,
       2., 7., 1., 5., 6., 1., 6., 6., 1., 3., 4., 4., 1., 4., 1., 6., 1.,
       7., 6.], dtype=float32)

In [73]:
accuracy = []
count = 0
for i,c in enumerate(y_test):
    if c == a[i]:
      accuracy.append(1)
      count += 1
    else:
      accuracy.append(0)
print('accuracy', count/len(accuracy))

accuracy 0.8275862068965517
